# Lab 3


In [1]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import numpy as np

# The Nim and Nimply classes

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects") # move

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

# Rule based agent using Nim-Sum


In [4]:
def nim_sum_l(nim):
  X = nim._rows[0]
  # X is the nim_sum(bitwise xor) of all heap sizes
  for i in range(1, len(nim._rows)):
    X = X ^ nim._rows[i]

  nim_sum_val = []
  # calculate the nim_sum between X and each heap size
  for i in nim._rows:
    val = i ^ X
    nim_sum_val.append(val)

  row = "false"
 
  for i in range(len(nim_sum_val)):
    if nim_sum_val[i] < nim._rows[i]:
      row = i
      break
  
  # reduce that heap to value nim_sum
  if (row != "false"):
    num_objects = nim._rows[row] - nim_sum_val[row]
    move = Nimply(row, num_objects)
  
  else:
    rand_row = random.randrange(0,len(nim._rows))

    while(nim._rows[rand_row] == 0):
      rand_row = random.randrange(0,len(nim._rows))

    if(nim._rows[rand_row] != 1):
      rand_obj = random.randrange(1, nim._rows[rand_row])
                                  
    else: rand_obj = 1
    move = Nimply(rand_row, rand_obj)
  return move

# Evolved Strategy

## Parameters
- How many rows that are still alive <br />
- How many of the inital sticks are left <br />
- <br />


## Rule
- Defence: If stick < 5% - take only 1 stick <br />
-  <br />

     0 the middle column
     pick one from the longest
     pick one from random rows
     zero the smallest row
     half the biggest row
     take k sticks if possible otherwise take the max <br />

     onerow-stategy: only one row - leave x
     2 rows: 1 rad = 1 element. ta x element från den andra raden
     2 rows: both has more than 1 element - take random
     else: take random random. 

     genome: dict 
     individ = {rule1: x, rule2:y, rule3:z, rule4:n}

In [5]:
MATCHES = 500


def strategy():
    if one_row:
        return one_row_start()

In [6]:
#Remove x element from the only row left
# 0 < x <= k
def one_row_strategy(state: Nim, individual) -> Nimply:
    row_counter = -1
    for row in state.rows: 
        row_counter += 1
        if row > 0:
            return Nimply(row_counter , min(row, individual[0]))


In [7]:
def two_rows_strategy1(nim, individual):
    return None

In [8]:
def count_rows_alive(nim):
    rows_alive = 0
    for i in nim._rows:
        if nim._rows == 0:
            count += 1
    return rows_alive

In [9]:
def zero_middle_column(nim):
    #find the row with the amount of sticks that are in the middle
    sorted_rows = sorted(nim._rows)

    middle_row = int(len(sorted_rows)/2) #make sure it is an even number

    obj = sorted_rows[middle_row]

    for i in nim._rows:
        if nim._rows[i] == obj:
            final_row = i

    move = Nimply(i, obj)
    
    return move


# MinMax


In [10]:
#player 0 - maximizing
#player 1 - minimizing

def min_max(nim):
    if state == 0:
        return None



In [11]:
def all_possible_moves(nim):
    possible_moves = []

    for row in range(len(nim._rows)):
        for j in range(nim._rows[row]):
            copy_row = deepcopy(row)

            new_move = copy_row[row] - j - 1
            possible_moves.append(new_move)

    return possible_moves
            

In [12]:
def evaluate(nim, player):
    if not nim: # if game is finished
        # player 0 loose if return -1
        return -1 if player == 0 else 1


# Sample strategies

In [13]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [14]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

In [15]:
def active_rows_index(state: Nim) -> list:
    active_rows_index = []
    count = 0
    for o in state.rows:
        if o > 0:
          active_rows_index.append(count)
        count += 1
    return active_rows_index

In [16]:
def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["active_rows_index"] = active_rows_index(state)
    cooked["rows_with_one_element"] = [(index, r) for index, r in enumerate(state.rows) if r == 1]
    

    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)
    
    #cooked["one_row_left"] = one_row_strategy(state)
    cooked["pure_random"] = pure_random(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [17]:
state = Nim(3)

ply = Nimply(2,4)
state.nimming(ply)
ply = Nimply(0,1)
state.nimming(ply)

data = cook_status(state)

print("rows_with_one_elem", data["rows_with_one_element"])

print("rdfjh",state.rows[data["active_rows_index"][0]])

#one_match = [(index, r) for index, r in enumerate(state.rows) if r == 1]



rows_with_one_elem [(2, 1)]
rdfjh 3


In [18]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        active_rows_number = data["active_rows_number"]
        rows_with_one_elem = data["rows_with_one_elem"]
        active_rows_index = data["active_rows_index"]

        #only one active row left
        if active_rows_number == 1: 
           
            #Rule 1 - if only one row left: leave x sticks
            last_row = state.rows[data["active_rows_index"][0]]
            if last_row > genome['Rule1']: #if the rule want to leave more sticks than exists in row
                ply = Nimply(last_row, 1)
            else:
                #add if you want to take more than K - take k
                #if (k <)
                elem_to_remove = last_row - genome["Rule1"] #total amout of elem in list - elem that will remain. 5 elem in list, want to have 3 left -> remove 5-3=2
                ply = Nimply(last_row, elem_to_remove)
        
        elif active_rows_number == 2:
            
            #rule 2
            #if both row has one elem, do rule2 either way
            if rows_with_one_elem: 
                logging.debug("Starting rule 1")
                if genome["Rule2"][0] == 0:  # take from row with one elem
                    elem = 1 # want to take the last elem in row
                    row = rows_with_one_elem[0][0] # looks like [(row,elem),(row,elem)] if two rows with single elem
                    ply = Nimply(row, 1)

                else: #take from the row with more than one element
                    row = [i for i in active_rows_index if state.rows[i] != 1][0]
                    if (genome['Rule2'][1] > row[1]): # if it wants to leave more elem than exists in row
                        elem = 1
                        ply = Nimply(row[0], 1)
                        
            for row in active_rows_number:
                if row[1] == 1: # check if only one elem left in the row
                    row_with_one_elem = row
                    

            if row_with_one_elem:
                # Rule 2
                #2 rows: 1 rad = 1 element. ta x element från den andra raden
                # rule2 = [0=row with one elem | 1=row with more than one , number of elements to leave]
                

            else:
                # Rule 3

            
            if genome["Rule2"][0] == 1:

            


        else:
            ply = Nimply(data["pure_random"], random.randint(1, state.rows[data["pure_random"]]))

        return ply

    return evolvable

IndentationError: expected an indented block (3472125087.py, line 37)

In [ ]:
#compute the fittness
def head2head(individual1: dict, individual2: dict, nim_size) -> int:
    won = 0
    NUM_MATCHES = 1 
    nim = Nim(nim_size)
    players = (make_strategy(individual1), make_strategy(individual2))

    while nim:
        ply = players[player](nim)
        nim.nimming(ply)
        player = 1 - player
    winner = 1 - player
    if winner == 0:
        individual1["fitness"] +=1
        #öka fitness på individ1 vid vinst
    else: 
        individual2["fitness"] +=1
        #öka fitness på individ2 vid vinst
   


In [ ]:
# calculate fitness for whole population
def calculate_fitness(state: Nim, population: list, nim_size):
    for p1 in population:
        for p2 in population:
            head2head(p1,p2, nim_size)


In [ ]:

def init_population(population_size: int, nim_size: int):
    population = []
    max_leave = (nim_size-1)*2 # last row of the table will have nim_size*2-1 objects

    while population_size:
        individual = {'Rule1': random.randint(0,max_leave), 'Rule2': [random.randint(0,1), max_leave], 'Rule3': random.randint(0, max_leave)}
        individual['fitness'] = 0
        population.append(individual)
        population_size -= 1
        
    return population

In [ ]:
def k_fittest_indv(population: list, k: int) -> list:
    return sorted(population, key=lambda indv: indv['fitness'], reverse=True)[:k]


In [ ]:
def clean_fitness(individuals: list):
    for indv in individuals:
        indv['fitness'] = 0

In [ ]:
# select k random individuals, return the one with best fitness
def tournament(population: list, k: int) -> dict:    
    contestors = random.sample(population, k)
    clean_fitness(contestors)
    
    for p1 in contestors:
        for p2 in contestors:
            if p1!=p2:
                head2head(p1, p2)
    best_contestor = sorted(contestors, key=lambda indv: indv['fitness'], reverse=True)[0]

    return best_contestor
    

In [ ]:
# take two parents and create one child
def crossover(parent1: dict, parent2: dict, mutation_prob: float) -> dict:
    rules_parent1 = [key for key in parent1.keys() if 'Rule' in key]
    rules_parent2 = [key for key in parent1.keys() if 'Rule' in key]

    child = {}

    for rule in rules_parent1:
        which_parent = random.randint(0,1)
        if which_parent == 0: # take rule from parent1
            child[rule] = rules_parent1[rule]
        else:
            child[rule] = rules_parent2[rule]
    
    child['fitness'] = 0
    rules = [key for key in child.keys() if 'Rule' in key]

    #mutation
    if random.random() < mutation_prob:
        rule = random.choice(rules)
        r1 = parent1[rule]
        r2 = parent2[rule]
        child[rule] = [int(d) for d in np.mean([r1, r2], axis=0).toList()]
    logging.INFO('Child: {child}')
        
    return child

     

# Evolution strategy described

1. Create inital population (with POPULATION_SIZE) where each player has the same set of rules but different parameters
2. Generate offspring where OFFSPRING_SIZE>>POPULATION_SIZE:
    - k individuals compete against each other and the fittest becomes a parent
    - Preform crossover between two parents and create one child
3. Play all children against each other to calculate fitness
4. Pick out the top l 
2. Evaluate:
    - Calculate fitness - all children plays agains each other (winner increase fitness)
3. Select the top k fittest children
4. Repeat 2-3 steps GENERATION amount of times


In [ ]:
OFFSPRING_SIZE = 200
K= 10
def create_offspring(population: list, mutation_prob: float) -> list:
    offspring = []
    for i in range(OFFSPRING_SIZE):
        parent1 = tournament(population=population,k=K)
        parent2 = tournament(population=population, k=K)

        child = crossover(parent1, parent2, mutation_prob=mutation_prob)

        offspring.append(child)

    return offspring

In [ ]:
def get_next_generation(offspring: list) -> list:
    for p1 in offspring:
        for p2 in offspring:
            calculate_fitness()
    
    best_k_offspring = sorted(offspring, key=lambda child: child['fitness'], reverse=True)[:POPULATION_SIZE]

    return best_k_offspring


In [ ]:
POPULATION_SIZE = 10
pop = init_population(2, POPULATION_SIZE)

print(pop)
print(POPULATION_SIZE)

<1 3 5>
[{'Rule1': 12, 'Rule2': [0, 18], 'Rule3': 18}, {'Rule1': 7, 'Rule2': [0, 18], 'Rule3': 14}]
10


In [ ]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

Oversimplified match

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

#strategy = (nim_sum_l({"p": 0.1}), optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = nim_sum_l(nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")


DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 6 19 21>
DEBUG:root:status: After player 1 -> <1 3 3 7 9 11 13 15 6 19 21>
DEBUG:root:status: After player 0 -> <1 3 3 1 9 11 13 15 6 19 21>
DEBUG:root:status: After player 1 -> <1 3 3 1 9 11 13 15 6 8 21>
DEBUG:root:status: After player 0 -> <1 3 3 1 9 11 13 15 6 8 14>
DEBUG:root:status: After player 1 -> <0 3 3 1 9 11 13 15 6 8 14>
DEBUG:root:status: After player 0 -> <0 2 3 1 9 11 13 15 6 8 14>
DEBUG:root:status: After player 1 -> <0 2 3 0 9 11 13 15 6 8 14>
DEBUG:root:status: After player 0 -> <0 2 2 0 9 11 13 15 6 8 14>
DEBUG:root:status: After player 1 -> <0 2 2 0 9 11 9 15 6 8 14>
DEBUG:root:status: After player 0 -> <0 2 2 0 9 11 9 11 6 8 14>
DEBUG:root:status: After player 1 -> <0 2 2 0 9 10 9 11 6 8 14>
DEBUG:root:status: After player 0 -> <0 2 2 0 8 10 9 11 6 8 14>
DEBUG:root:status: After player 1 -> <0 1 2 0 8 10 9 11 6 8 14>
DEBUG:root:status: After 